# EXP-009: Стекинг (OOF + мета-модель)

**Результат: Public LB 0.8444** (лучший результат, +0.0032 к EXP-007b)

## Архитектура
- **Уровень 1 (L1)**: XGBoost 41 модель × 5-fold OOF + per-target feature selection (cumulative gain 95%)
- **Уровень 2 (L2)**: XGBoost мета-модель (depth=2) на 41 OOF-фиче → учит корреляции между таргетами

## Пайплайн
1. Загрузка данных → numpy float32
2. Генерация OOF предсказаний (750k × 41) с отбором фичей
3. Full train + test inference с отобранными фичами
4. Мета-модель на OOF → финальный сабмит

## Ключевой инсайт
Мета-модель видит связи между таргетами: если клиент покупает продукт A (частый),
то продукт B (редкий) тоже вероятен. Слабые таргеты получают сигнал от частых связанных.
На быстром тесте (100k): target_5_2 +0.081, target_2_5 +0.071.

## Шаг 1: Загрузка и подготовка данных

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import time
import gc
import json
import warnings

warnings.filterwarnings("ignore")

# === ПУТИ ===
# Для Spark:
DATA = '/home/masked/projects/data_fusion/data/raw'
# Для Colab:
# DATA = '/content/drive/MyDrive/data_fusion'

print("1. Загрузка обучающей выборки...")
train_main = pd.read_parquet(f'{DATA}/train_main_features.parquet')
train_extra = pd.read_parquet(f'{DATA}/train_extra_features.parquet')
train_target = pd.read_parquet(f'{DATA}/train_target.parquet')

X_train_df = train_main.merge(train_extra, on='customer_id', how='left')
X_train_df = X_train_df.drop(columns=['customer_id'])

y_train_df = train_target.drop(columns=['customer_id'])
target_columns = y_train_df.columns.tolist()

print("2. Загрузка тестовой выборки...")
test_main = pd.read_parquet(f'{DATA}/test_main_features.parquet')
test_extra = pd.read_parquet(f'{DATA}/test_extra_features.parquet')

X_test_df = test_main.merge(test_extra, on='customer_id', how='left')
test_customer_ids = test_main['customer_id'].values  # Для сабмита
X_test_df = X_test_df.drop(columns=['customer_id'])

feature_columns = X_train_df.columns.tolist()

print("3. Конвертация в numpy.float32 (для GPU)...")
start = time.time()

X_train_np = X_train_df.astype(np.float32).values
y_train_np = y_train_df.values
X_test_np = X_test_df.astype(np.float32).values

print(f"Конвертация завершена за {time.time() - start:.1f} сек.")
print(f"X_train: {X_train_np.shape} | X_test: {X_test_np.shape} | y_train: {y_train_np.shape}")

del train_main, train_extra, train_target, test_main, test_extra
del X_train_df, y_train_df, X_test_df
gc.collect()
print("Готово!")

## Шаг 2: Уровень 1 — Генерация OOF предсказаний

Для каждого из 41 таргета:
1. **Черновик** (100 iter на всех фичах) → извлекаем feature importance → отбираем топ-95% cumulative gain
2. **Чистовик** (500 iter на отобранных фичах) → 5-fold StratifiedKFold → OOF предсказания

Результат: матрица `xgb_oof_train.npy` (750000, 41) — честные предсказания для каждого клиента.

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

N_FOLDS = 5
N_ROUNDS_DRAFT = 100   # Черновик для отбора фичей
N_ROUNDS_FINAL = 500   # Чистовик для OOF
GAIN_THRESHOLD = 0.95  # Порог cumulative gain

base_params = {
    'objective': 'binary:logistic', 'eval_metric': 'auc',
    'learning_rate': 0.05, 'max_depth': 6,
    'subsample': 0.8, 'colsample_bytree': 0.8,
    'tree_method': 'hist', 'device': 'cuda', 'seed': 42, 'verbosity': 0,
}

oof_predictions = np.zeros_like(y_train_np, dtype=np.float32)
oof_aucs = []
best_features_dict = {}

# Мастер-матрица для быстрых черновиков (строится один раз)
print("Выравнивание памяти (ascontiguousarray)...")
X_train_np = np.ascontiguousarray(X_train_np)

print("Построение мастер-матрицы для черновиков (1-2 мин)...")
dtrain_full_draft = xgb.QuantileDMatrix(X_train_np)
print("Мастер-матрица готова!")

print(f"Начинаем OOF цикл для {len(target_columns)} таргетов...")
start_total = time.time()

for t_idx, target_col in enumerate(target_columns):
    start_target = time.time()
    y_target = y_train_np[:, t_idx]
    n_pos = int(y_target.sum())

    # Защита: если позитивов меньше фолдов — пропускаем
    current_folds = min(N_FOLDS, n_pos) if n_pos > 0 else 0
    if current_folds < 2:
        print(f"[{t_idx+1:2d}/41] {target_col:<15} ПРОПУСК (pos: {n_pos})")
        oof_predictions[:, t_idx] = 0.0001
        oof_aucs.append(0.5)
        best_features_dict[target_col] = feature_columns
        continue

    skf = StratifiedKFold(n_splits=current_folds, shuffle=True, random_state=42)

    # Адаптивный min_child_weight: для редких таргетов разрешаем мелкие сплиты
    current_min_child = 5 if n_pos > 500 else 1
    current_params = base_params.copy()
    current_params['min_child_weight'] = current_min_child

    # --- ШАГ 1: Черновик → отбор фичей по cumulative gain 95% ---
    dtrain_full_draft.set_label(y_target)
    model_draft = xgb.train(current_params, dtrain_full_draft,
                            num_boost_round=N_ROUNDS_DRAFT, verbose_eval=False)

    imp = model_draft.get_score(importance_type='gain')
    imp_sorted = sorted(imp.items(), key=lambda x: x[1], reverse=True)

    selected_indices = []
    if not imp_sorted:
        selected_indices = list(range(X_train_np.shape[1]))
    else:
        total_gain = sum(v for _, v in imp_sorted)
        cum_gain = 0
        for feat, gain in imp_sorted:
            cum_gain += gain
            feat_idx = int(feat[1:])  # 'f123' -> 123
            selected_indices.append(feat_idx)
            if cum_gain / total_gain >= GAIN_THRESHOLD:
                break

        if len(selected_indices) < 10:
            selected_indices = [int(f[1:]) for f, _ in imp_sorted[:10]]

    best_features_dict[target_col] = [feature_columns[i] for i in selected_indices]

    # --- ШАГ 2: Чистовик — K-Fold OOF на отобранных фичах ---
    X_target_np = X_train_np[:, selected_indices].copy(order='C')

    for train_idx, val_idx in skf.split(X_target_np, y_target):
        dtrain = xgb.QuantileDMatrix(X_target_np[train_idx], label=y_target[train_idx])
        dval = xgb.QuantileDMatrix(X_target_np[val_idx], label=y_target[val_idx], ref=dtrain)

        model = xgb.train(current_params, dtrain,
                          num_boost_round=N_ROUNDS_FINAL, verbose_eval=False)
        oof_predictions[val_idx, t_idx] = model.predict(dval)

    # Итоги таргета
    try:
        auc = roc_auc_score(y_target, oof_predictions[:, t_idx])
    except ValueError:
        auc = 0.5

    oof_aucs.append(auc)
    elapsed_target = (time.time() - start_target) / 60
    print(f"[{t_idx+1:2d}/41] {target_col:<15} OOF AUC: {auc:.4f} | pos: {n_pos:<6} | "
          f"Фичей: {len(selected_indices):<4} | {elapsed_target:.1f} мин")

    # Чекпоинт каждые 5 таргетов
    if (t_idx + 1) % 5 == 0:
        np.save(f'{DATA}/xgb_oof_checkpoint.npy', oof_predictions)
        with open(f'{DATA}/xgb_best_features_checkpoint.json', 'w') as f:
            json.dump(best_features_dict, f)

# === Итоги ===
elapsed_total = (time.time() - start_total) / 60
print(f"\nГотово! Общее время: {elapsed_total:.1f} мин")
print(f"Базовый OOF Macro AUC (XGBoost): {np.mean(oof_aucs):.4f}")

np.save(f'{DATA}/xgb_oof_train.npy', oof_predictions)
with open(f'{DATA}/xgb_best_features.json', 'w') as f:
    json.dump(best_features_dict, f)
print("OOF-матрица и словарь фичей сохранены!")

## Шаг 3: Full Train + Test Inference

Обучаем XGBoost на **всех** 750k для каждого таргета с отобранными фичами,
предсказываем на test (250k).

Результат: `xgb_test_preds.npy` (250000, 41) — базовые предсказания уровня L1.

In [ ]:
N_ROUNDS_FINAL = 500

# Загружаем отобранные фичи (если ядро перезапускалось)
with open(f'{DATA}/xgb_best_features.json', 'r') as f:
    best_features_dict = json.load(f)

# Выравниваем память для test
print("Выравнивание памяти...")
X_train_np = np.ascontiguousarray(X_train_np)
X_test_np = np.ascontiguousarray(X_test_np)

test_predictions = np.zeros((X_test_np.shape[0], len(target_columns)), dtype=np.float32)

print(f"Начинаем обучение и inference для {len(target_columns)} таргетов...")
start_total = time.time()

for t_idx, target_col in enumerate(target_columns):
    start_target = time.time()
    y_target = y_train_np[:, t_idx]
    n_pos = int(y_target.sum())

    if n_pos < 2:
        print(f"[{t_idx+1:2d}/41] {target_col:<15} ПРОПУСК (pos: {n_pos})")
        test_predictions[:, t_idx] = 0.0001
        continue

    current_min_child = 5 if n_pos > 500 else 1
    current_params = base_params.copy()
    current_params['min_child_weight'] = current_min_child

    # Индексы отобранных фичей
    selected_feature_names = best_features_dict[target_col]
    selected_indices = [feature_columns.index(name) for name in selected_feature_names]

    # C-contiguous срезы для скорости
    X_train_target = X_train_np[:, selected_indices].copy(order='C')
    X_test_target = X_test_np[:, selected_indices].copy(order='C')

    dtrain = xgb.QuantileDMatrix(X_train_target, label=y_target)
    dtest = xgb.QuantileDMatrix(X_test_target, ref=dtrain)

    model = xgb.train(current_params, dtrain,
                      num_boost_round=N_ROUNDS_FINAL, verbose_eval=False)
    test_predictions[:, t_idx] = model.predict(dtest)

    elapsed_target = (time.time() - start_target) / 60
    if (t_idx + 1) % 5 == 0 or t_idx == 0:
        print(f"[{t_idx+1:2d}/41] {target_col:<15} Готово | "
              f"Фичей: {len(selected_indices):<4} | {elapsed_target:.1f} мин")

elapsed_total = (time.time() - start_total) / 60
print(f"\nInference завершён! Общее время: {elapsed_total:.1f} мин")

np.save(f'{DATA}/xgb_test_preds.npy', test_predictions)
print("Тестовые предсказания сохранены!")

## Шаг 4: Уровень 2 — Мета-модель (Стекинг)

Мета-модель (XGBoost depth=2) обучается предсказывать каждый таргет
на основе **41 OOF-предсказания**. Она учит корреляции:
- Группы 3-7 коррелируют (пакетные покупки)
- target_10_1 — антагонист (отрицательная корреляция)
- Редкие таргеты получают сигнал от частых связанных

Результат: `submission.parquet` — финальный сабмит.

In [ ]:
# Загружаем OOF и test предсказания (если ядро перезапускалось)
X_meta_train = np.load(f'{DATA}/xgb_oof_train.npy')
X_meta_test = np.load(f'{DATA}/xgb_test_preds.npy')

# Загружаем таргеты
train_target = pd.read_parquet(f'{DATA}/train_target.parquet')
y_train = train_target.drop(columns=['customer_id']).values
target_columns = train_target.drop(columns=['customer_id']).columns.tolist()

# ID клиентов для сабмита
test_main = pd.read_parquet(f'{DATA}/test_main_features.parquet')
test_customer_ids = test_main['customer_id'].values
del train_target, test_main
gc.collect()

print(f"OOF train: {X_meta_train.shape} | Test preds: {X_meta_test.shape}")

# Параметры мета-модели: неглубокие деревья, чтобы не переобучиться на 41 фиче
meta_params = {
    'objective': 'binary:logistic', 'eval_metric': 'auc',
    'max_depth': 2, 'learning_rate': 0.05, 'n_estimators': 100,
    'tree_method': 'hist', 'device': 'cuda', 'random_state': 42,
}

final_test_preds = np.zeros_like(X_meta_test)
meta_aucs = []

print(f"\nОбучение мета-модели (L2) для {len(target_columns)} таргетов...")
start = time.time()

for i, col in enumerate(target_columns):
    clf = xgb.XGBClassifier(**meta_params)
    clf.fit(X_meta_train, y_train[:, i])

    final_test_preds[:, i] = clf.predict_proba(X_meta_test)[:, 1]

    # Train AUC мета-модели (завышен, т.к. не OOF для L2)
    oof_meta_preds = clf.predict_proba(X_meta_train)[:, 1]
    auc = roc_auc_score(y_train[:, i], oof_meta_preds)
    meta_aucs.append(auc)

    if (i + 1) % 5 == 0 or i == 0:
        print(f"[{i+1:2d}/41] {col:<15} Meta-AUC: {auc:.4f}")

print(f"\nСредний Meta-AUC: {np.mean(meta_aucs):.4f}")
print(f"Время: {(time.time() - start)/60:.1f} мин")

# === Формирование сабмита ===
submission = pd.DataFrame({'customer_id': test_customer_ids})
for i, col in enumerate(target_columns):
    # predict_ без target_ — формат платформы
    col_name = col.replace('target_', '')
    submission[f'predict_{col_name}'] = final_test_preds[:, i].astype(np.float64)

submission.to_parquet('submission.parquet', index=False)
print(f"\nСабмит готов! {submission.shape}")
print(submission.head(2))